In [1]:
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    feature_columns = [col for col in data.columns if col not in ['id','target']]
    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(len(x))
            if(len(x)>total_records*0.01):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return(999,);
    if total_rule/total_records<0.4:
        return (999,);

    avg_confidence = np.sum(rule_confidences) / count +0*(entropy_D-entropy_DB)
    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    # create individual
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    return hof



def bestIndividual(hof):

    def SVM(trainX,trainY,testX,testY):
        scaler = StandardScaler()
        scaler.fit(trainX)
        trainX3 = scaler.transform(trainX)
        testX3 = scaler.transform(testX)
        clf = svm.SVC(C=10, gamma=0.8, max_iter=200)
        clf.fit(trainX3, trainY)
        test_result = clf.predict(testX3)
        precision = np.sum(test_result == testY)/testY.shape[0]
        return precision
    maxaccuracy=0.0
    for ind in hof:
        df_train = pd.read_csv('data/Soybean/soybean-large_train_fillna.csv')
        df_test = pd.read_csv('data/Soybean/soybean-large_test_fillna.csv')
        feature_columns = [col for col in df_train.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]

        trainX=df_train[selected_features]
        trainY=df_train['target']
        testX=df_test[selected_features]
        testY=df_test['target']
        score=SVM(trainX,trainY,testX,testY)
        if(score>maxaccuracy):
            maxaccuracy=score
            print(maxaccuracy)
            best_feature=selected_features
            print(best_feature)
    return maxaccuracy,best_feature

In [2]:
if __name__ == '__main__':
    # get dataframe path, population number and generation number from command-line argument
    # dataframePath, n_pop, n_gen = getArguments()
    n_pop=40
    n_gen=80
    # read dataframe from csv
    df = pd.read_csv('data/Soybean/soybean-large_train_fillna.csv')
    # encode labels column to numbers
    y = df['target']
    feature_columns = [ col for col in df.columns if col not in ['id','target']]
    X = df[feature_columns]
    for col in feature_columns:
        df[col] = pd.qcut(df[col], q=5, labels=False, duplicates='drop')
    # get accuracy with all features
    individual = [1 for i in range(len(X.columns))]
    getFitness(df,'target',individual)
    print("Accuracy with all features: \t" +
          str(getFitness(df,'target',individual)) + "\n")

    # apply genetic algorithm
    hof = geneticAlgorithm(df,'target',n_pop, n_gen)

Accuracy with all features: 	(999,)



C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg    	min     
0  	40    	549.688	0.330775
1  	18    	325.012	0.330775
2  	27    	100.293	0.236803
3  	19    	25.3327	0.184593
4  	20    	25.273 	0.170704
5  	24    	50.1846	0.170704
6  	20    	25.158 	0.170704
7  	23    	0.18288	0.170704
8  	31    	25.1456	0.170704
9  	21    	25.1469	0.170704
10 	21    	25.1498	0.170704
11 	26    	0.178877	0.170704
12 	23    	25.1533 	0.164447
13 	21    	25.1481 	0.164447
14 	15    	75.0905 	0.164447
15 	31    	50.1095 	0.164447
16 	30    	25.1415 	0.164447
17 	19    	0.171916	0.164447
18 	18    	50.1064 	0.164447
19 	30    	50.1097 	0.164447
20 	31    	25.138  	0.164447
21 	27    	25.1382 	0.164447
22 	24    	50.1199 	0.164447
23 	20    	0.168002	0.164447
24 	17    	0.164604	0.164447
25 	25    	25.1375 	0.164447
26 	20    	25.1486 	0.164447
27 	29    	25.1431 	0.164447
28 	29    	25.1414 	0.164447
29 	26    	0.167606	0.164447
30 	20    	25.1474 	0.164447
31 	20    	25.1371 	0.164447
32 	31    	25.1384 	0.164447
33 	25    	25.1376 	0.1644

In [3]:
accuracy,individual = bestIndividual(hof)
print(accuracy)
print(individual)
    # select the best individual

    # print('Best Accuracy: \t' + str(accuracy))
    # print('Number of Features in Subset: \t' + str(individual.count(1)))
    # print('Individual: \t\t' + str(individual))
    # print('Feature Subset\t: ' + str(header))
    #
    # print('\n\ncreating a new classifier with the result')

    # read dataframe from csv one more time
    # df = pd.read_csv(dataframePath, sep=',')
    #
    # # with feature subset
    # X = df[header]
    #
    # clf = LogisticRegression()
    #
    # scores = cross_val_score(clf, X, y, cv=5)
    # print("Accuracy with Feature Subset: \t" + str(avg(scores)) + "\n")


0.6941489361702128
['f2', 'f4', 'f9', 'f10', 'f11', 'f13', 'f14', 'f15', 'f16', 'f17', 'f19', 'f22', 'f23', 'f24', 'f25', 'f29', 'f31', 'f34']
0.6968085106382979
['f2', 'f4', 'f9', 'f10', 'f11', 'f14', 'f15', 'f16', 'f18', 'f19', 'f20', 'f23', 'f24', 'f25', 'f29', 'f31', 'f34']
0.7021276595744681
['f2', 'f4', 'f9', 'f10', 'f11', 'f14', 'f15', 'f16', 'f17', 'f18', 'f20', 'f21', 'f22', 'f23', 'f24', 'f29', 'f31']
0.7101063829787234
['f2', 'f4', 'f9', 'f10', 'f11', 'f13', 'f14', 'f15', 'f16', 'f20', 'f23', 'f24', 'f25', 'f29', 'f31', 'f34', 'f36']
0.723404255319149
['f2', 'f4', 'f9', 'f10', 'f11', 'f13', 'f14', 'f15', 'f16', 'f17', 'f19', 'f22', 'f23', 'f24', 'f27', 'f28', 'f29', 'f34', 'f36']
0.7473404255319149
['f2', 'f4', 'f9', 'f10', 'f11', 'f12', 'f14', 'f15', 'f16', 'f19', 'f20', 'f22', 'f23', 'f24', 'f29', 'f31']
0.7526595744680851
['f2', 'f3', 'f4', 'f10', 'f11', 'f14', 'f15', 'f16', 'f18', 'f19', 'f22', 'f23', 'f24', 'f29', 'f31', 'f32', 'f34', 'f36']
0.7632978723404256
['f2', 'f

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


0.8829787234042553
['f2', 'f4', 'f14', 'f15', 'f16', 'f17', 'f19', 'f21', 'f23', 'f26', 'f29', 'f34', 'f36']


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
